# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [30]:
!apt-get update && apt-get install -y python3-opencv

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2564 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]   
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3002 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1468 kB]
Get:14 http://security.ubuntu.com

In [31]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [32]:
!pip install sklearn

In [1]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold

import torchvision.transforms as transforms

## Set Arguments & hyperparameters

In [2]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [3]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128

####
k = 5

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [4]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.8)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.8):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']


## Model

In [5]:
# import torch.nn.functional as F

# class custom_CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(custom_CNN, self).__init__()
        
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        
#         self.pool = nn.MaxPool2d(kernel_size=2)
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
#         self.fc1 = nn.Linear(in_features=25*29*29, out_features=128)
#         self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
#         x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
#         x = torch.flatten(x,1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        
#         output = self.softmax(x)
        
#         return output

In [10]:
# import torchvision.models as models

# class efficientnet_b0(nn.Module):
#     def __init__(self, num_classes):
#         super(efficientnet_b0, self).__init__()
#         self.model = models.efficientnet_b0(pretrained=False)
#         self.model.classifier = nn.Sequential(
#             nn.Dropout(p=0.2, inplace=True),
#             nn.Linear(1280, num_classes),
#         )
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, input_img):
#         x = self.model(input_img)
#         x = self.softmax(x)
#         return x 

In [8]:
# import torchvision.models as models

# class densenet161(nn.Module):
#     def __init__(self, num_classes):
#         super(densenet161, self).__init__()
#         self.model = models.densenet161(pretrained=False)
#         self.model.classifier = nn.Linear(in_features=2208, out_features=num_classes, bias=True)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, input_img):
#         x = self.model(input_img)
#         x = self.softmax(x)
#         return x 

In [26]:
import torchvision.models as models

class densenet201(nn.Module):
    def __init__(self, num_classes):
        super(densenet201, self).__init__()
        self.model = models.densenet201(pretrained=False)
        self.model.classifier = nn.Linear(in_features=1920, out_features=num_classes, bias=True)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_img):
        x = self.model(input_img)
        x = self.softmax(x)
        return x

## Utils
### EarlyStopper

In [27]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [28]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, fold, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
            
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Fold{fold +1} /Epoch{epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, fold, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Fold{fold + 1} Epoch/{epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [29]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [30]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
###
dataset = ConcatDataset([train_dataset, validation_dataset])
splits=KFold(n_splits=k,shuffle=True,random_state=42)
####
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 516 Val set samples: 130


In [12]:
# for i in splits.split(np.arange(len(dataset))):
#     print(i)

#### Load model and other utils

In [31]:
# Load Model
model = densenet201(NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(dataset))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

# K Fold

In [ ]:
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)
    

    for epoch_index in tqdm(range(EPOCHS)):


        trainer.train_epoch(fold, train_loader, epoch_index)
        trainer.validate_epoch(fold, test_loader, epoch_index)

        # early_stopping check
        early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

        if early_stopper.stop:
            print('Early stopped')
            break

        if early_stopper.save_model:           
            check_point = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict()
            }
            torch.save(check_point, 'best.pt')

  0% 0/10 [00:00<?, ?it/s]

Fold 1
Fold1 /Epoch0, Train loss: 0.7427005842328072, Acc: 0.5193798449612403, F1-Macro: 0.34543222506393867


 10% 1/10 [01:05<09:51, 65.76s/it]

Fold1 Epoch/0, Val loss: 0.8346105962991714, Acc: 0.5461538461538461, F1-Macro: 0.35323383084577115
Fold1 /Epoch1, Train loss: 0.7440792731940746, Acc: 0.5213178294573644, F1-Macro: 0.3462893720475773
Fold1 Epoch/1, Val loss: 0.8263923078775406, Acc: 0.5384615384615384, F1-Macro: 0.35
Validation loss decreased 0.8346105962991714 -> 0.8263923078775406


 20% 2/10 [02:08<08:37, 64.72s/it]

...

Fold3 /Epoch9, Train loss: 0.07427348115015775, Acc: 0.988394584139265, F1-Macro: 0.9883527079390827
Fold3 Epoch/9, Val loss: 0.25768998079001904, Acc: 0.9069767441860465, F1-Macro: 0.9069264069264068
Early stopping counter 9/10


100% 10/10 [11:35<00:00, 69.52s/it]
  0% 0/10 [00:00<?, ?it/s]

Fold 4
Fold4 /Epoch0, Train loss: 0.2511172275990248, Acc: 0.9052224371373307, F1-Macro: 0.9048107525184776


  0% 0/10 [01:00<?, ?it/s]
  0% 0/10 [00:00<?, ?it/s]

Fold4 Epoch/0, Val loss: 0.07686681768313974, Acc: 0.9844961240310077, F1-Macro: 0.9842836257309941
Early stopping counter 10/10
Early stopped
Fold 5


### epoch 단위 학습 진행

In [17]:
# for epoch_index in enumerate(tqdm(range(EPOCHS)):

#     trainer.train_epoch(train_dataloader, epoch_index)
#     trainer.validate_epoch(validation_dataloader, epoch_index)

#     # early_stopping check
#     early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

#     if early_stopper.stop:
#         print('Early stopped')
#         break

#     if early_stopper.save_model:
#         check_point = {
#             'model': model.state_dict(),
#             'optimizer': optimizer.state_dict(),
#             'scheduler': scheduler.state_dict()
#         }
#         torch.save(check_point, 'best.pt')


  0% 0/10 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7150068382422129, Acc: 0.5679862306368331, F1-Macro: 0.5596371553413597


 10% 1/10 [00:43<06:31, 43.47s/it]

Epoch 0, Val loss: 1.0537893772125244, Acc: 0.6153846153846154, F1-Macro: 0.5905769715293524
Epoch 1, Train loss: 0.6220975004964404, Acc: 0.6764199655765921, F1-Macro: 0.6764190069913497
Epoch 1, Val loss: 0.8865877091884613, Acc: 0.5076923076923077, F1-Macro: 0.3627450980392157
Validation loss decreased 1.0537893772125244 -> 0.8865877091884613


 20% 2/10 [01:26<05:47, 43.39s/it]

...

 80% 8/10 [05:43<01:25, 42.70s/it]

Epoch 7, Val loss: 0.4742511548101902, Acc: 0.7692307692307693, F1-Macro: 0.7636363636363637
Validation loss decreased 0.4888179078698158 -> 0.4742511548101902
Epoch 8, Train loss: 0.352601816256841, Acc: 0.8605851979345955, F1-Macro: 0.8588727452656202
Epoch 8, Val loss: 0.4510917328298092, Acc: 0.7846153846153846, F1-Macro: 0.7833333333333333
Validation loss decreased 0.4742511548101902 -> 0.4510917328298092


 90% 9/10 [06:27<00:42, 42.95s/it]

Epoch 9, Train loss: 0.33472725252310437, Acc: 0.8726333907056799, F1-Macro: 0.8720568979883347
Epoch 9, Val loss: 0.44625740870833397, Acc: 0.7846153846153846, F1-Macro: 0.7841555977229602
Validation loss decreased 0.4510917328298092 -> 0.44625740870833397


100% 10/10 [07:09<00:00, 42.98s/it]


## Inference
### 모델 로드

In [18]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [19]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [20]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


In [21]:
len(test_dataset)

100

### 추론 진행

In [22]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

<All keys matched successfully>

In [23]:
model

densenet161(
  (model): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(144, eps=1e-05, momentum=

In [24]:
# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:02,  2.69s/it]

tensor([[9.9914e-01, 8.6040e-04],
        [9.3875e-01, 6.1253e-02],
        [7.2528e-01, 2.7472e-01],
        [4.9199e-01, 5.0801e-01],
        [9.9941e-01, 5.8897e-04],
        [2.2381e-03, 9.9776e-01],
        [4.0764e-03, 9.9592e-01],
        [1.4203e-01, 8.5797e-01],
        [9.8685e-01, 1.3153e-02],
        [2.0924e-03, 9.9791e-01],
        [9.9482e-01, 5.1792e-03],
        [9.3365e-01, 6.6353e-02],
        [9.9949e-01, 5.1246e-04],
        [4.6101e-01, 5.3899e-01],
        [8.8111e-01, 1.1889e-01],
        [9.8074e-01, 1.9263e-02],
        [7.1969e-01, 2.8031e-01],
        [9.9944e-01, 5.5629e-04],
        [8.4606e-01, 1.5394e-01],
        [9.9798e-01, 2.0173e-03],
        [9.9804e-01, 1.9580e-03],
        [9.8610e-01, 1.3897e-02],
        [8.6028e-01, 1.3972e-01],
        [8.5035e-01, 1.4965e-01],
        [9.7851e-04, 9.9902e-01],
        [9.9562e-01, 4.3769e-03],
        [9.9464e-01, 5.3586e-03],
        [1.7690e-01, 8.2310e-01],
        [2.3516e-02, 9.7648e-01],
        [4.273

2it [00:05,  2.85s/it]

tensor([[8.0640e-01, 1.9360e-01],
        [9.8737e-01, 1.2627e-02],
        [9.9123e-01, 8.7693e-03],
        [1.1274e-02, 9.8873e-01],
        [3.5710e-01, 6.4290e-01],
        [8.2763e-02, 9.1724e-01],
        [3.5589e-03, 9.9644e-01],
        [9.0481e-01, 9.5191e-02],
        [5.2442e-04, 9.9948e-01],
        [7.9600e-01, 2.0400e-01],
        [6.9956e-01, 3.0044e-01],
        [1.9566e-03, 9.9804e-01],
        [2.6712e-01, 7.3288e-01],
        [9.8614e-01, 1.3860e-02],
        [6.8384e-02, 9.3162e-01],
        [2.0698e-01, 7.9302e-01],
        [9.7225e-03, 9.9028e-01],
        [9.9985e-01, 1.5208e-04],
        [3.4690e-01, 6.5310e-01],
        [2.5228e-03, 9.9748e-01],
        [1.7010e-01, 8.2990e-01],
        [8.6156e-01, 1.3844e-01],
        [1.8574e-02, 9.8143e-01],
        [5.3056e-01, 4.6944e-01],
        [7.4469e-01, 2.5531e-01],
        [1.8442e-03, 9.9816e-01],
        [6.0988e-01, 3.9012e-01],
        [8.3839e-01, 1.6161e-01],
        [9.6630e-03, 9.9034e-01],
        [9.874

4it [00:09,  2.26s/it]

tensor([[9.6927e-02, 9.0307e-01],
        [2.5529e-03, 9.9745e-01],
        [7.4387e-04, 9.9926e-01],
        [9.1563e-01, 8.4366e-02],
        [1.5205e-01, 8.4795e-01],
        [9.9648e-01, 3.5231e-03],
        [9.2908e-01, 7.0916e-02],
        [9.9881e-01, 1.1883e-03],
        [9.6086e-01, 3.9142e-02],
        [9.9915e-01, 8.4809e-04],
        [4.0098e-05, 9.9996e-01],
        [6.4274e-01, 3.5726e-01],
        [9.9974e-01, 2.6495e-04],
        [9.9953e-01, 4.6974e-04],
        [4.3393e-02, 9.5661e-01],
        [2.6261e-02, 9.7374e-01],
        [2.0469e-04, 9.9980e-01],
        [1.4122e-02, 9.8588e-01],
        [9.4505e-01, 5.4954e-02],
        [5.3405e-01, 4.6595e-01],
        [4.7794e-02, 9.5221e-01],
        [1.2612e-01, 8.7388e-01],
        [9.8218e-01, 1.7824e-02],
        [1.0751e-01, 8.9249e-01],
        [9.9690e-01, 3.1016e-03],
        [2.1478e-01, 7.8522e-01],
        [3.4888e-01, 6.5112e-01],
        [4.9273e-01, 5.0727e-01],
        [1.3538e-02, 9.8646e-01],
        [1.173

### 결과 저장

In [25]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('densenet161.csv', index=False)